In [2]:
import urllib
import os
import math
from bs4 import BeautifulSoup as BS
import re
import psycopg2 as pg

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.mkdir(path)
        
def get_links(topic_link, num): # Возвращает ссылки статей подраздела
    pages = math.ceil(num/20)
    links = []
    for page in range(pages):
        content = BS(urllib.request.urlopen(f'https://cyberleninka.ru{topic_link}/{page+1}').read(), 'html.parser')
        for article in content.body.find_all('li'):
            for link in article.find_all('a'):
                if (re.match('/article/n', link['href'])):
                    links.append(link['href'])
    return links[0:num]

site_url = "https://cyberleninka.ru"
dir_root = "D:\\БИБЛИОТЕКА\\"
num_articles = 150 # Кол-во статей из одного раздела

main_page = BS(urllib.request.urlopen(site_url).read(), 'html.parser')
for letter in main_page.find_all('li', 'letter'): # Идём по разделам
    dir_letter = os.path.join(dir_root, '{}'.format(re.sub('^\W*', '', letter.get_text())))
    create_dir(dir_letter)
    
    topics = []
    for topic in letter.find_next_siblings(): # Идём по подразделам
        if (not topic.has_attr('class')):
            for a in topic.find_all('a'):
                dir_topic = os.path.join(dir_letter, '{}'.format(re.sub('^\W*', '', a.get_text())))
                create_dir(dir_topic)
                topics.append(a) # Ссылки на страницы со статьями подразделов
        else:
            break
            
    num_articles_on_topic = int(num_articles/len(topics)) # Кол-во статей на одну тему
    for topic in topics: # Идём по ссылкам на подразделы
        links = get_links(topic['href'], num_articles_on_topic)
        n = 1
        for link in links:
            file = site_url + link + '/pdf'
            name = link.split('/')[-1]
            path = os.path.join(dir_root, '{}'.format(re.sub('^\W*', '', letter.get_text())), '{}'.format(re.sub('^\W*', '', topic.get_text())), name) + '.pdf'
            if not os.path.exists(path):
                urllib.request.urlretrieve(file, path)
                print(f'DOWNLOADED:\n{n}. {path}')
            else:
                print(f'FILE EXISTS:\n{n}. {path}')
            n+=1

In [ ]:
sections = ['Гуманитарные науки', 'Естественные и точные науки', 'Медицинские науки', 'Сельскохозяйственные науки', 'Социальные науки', 'Техника и технологии']

dir_train = os.path.join(dir_root, 'Обучающий набор')
dir_test = os.path.join(dir_root, 'Тестовый набор')

for s in sections:
    os.rename(os.path.join(dir_root, s), os.path.join(dir_root, dir_train, s))

In [ ]:
#paths_train = []
#paths_test = []
#num_articles = []
test_proportion = 0.3

for s in sections:
    path = os.path.join(dir_root, dir_train, s)
    num_topics = len(os.listdir(path))
    for topic in os.listdir(os.path.join(dir_root, dir_train, s)):
        path = os.path.join(dir_root, dir_train, s, topic)
        num_articles = int(len(os.listdir(path)) * test_proportion)
        for file in os.listdir(path)[0:num_articles]:
            os.rename(os.path.join(path, file), os.path.join(dir_root, dir_test, s, topic, file))
        #paths_train.append((os.path.join(dir_root, dir_train, s, topic), num_articles))
        #paths_test.append(os.path.join(dir_root, dir_test, s, topic))
        
#for topic in paths_train:
#    print(topic)
#    for file in os.listdir(topic):
#        print(file)

In [35]:
sections = os.listdir('D:/БИБЛИОТЕКА/Обучающий набор/')
topics = {}
for section in sections:
   topic_list = os.listdir(f'D:/БИБЛИОТЕКА/Обучающий набор/{section}')
   topics[section] = topic_list

num_labels = len(sections)

section_map = {}
for i in range(len(sections)):
   section_map[i] = sections[i]

In [6]:
def load_files(dataset, topics, section_map):
   files_type = {'train': 'Обучающий набор',
                 'test': 'Тестовый набор'}
   if (dataset in files_type):
       files = []
       for i in section_map:
           for topic in topics[section_map[i]]:
               for file in os.listdir(f'D:/БИБЛИОТЕКА/{files_type[dataset]}/{section_map[i]}/{topic}'):
                   files.append((f'D:/БИБЛИОТЕКА/{files_type[dataset]}/{section_map[i]}/{topic}/{file}', i))
       return files
   else:
       print('Указан неверный тип данных в dataset')

In [21]:
# cur.close()
# conn.close()
conn = pg.connect('dbname=articles user=postgres password=1234')
cur = conn.cursor()

In [17]:
cur.execute('CREATE TABLE classes (id integer PRIMARY KEY, class_name varchar(40) NOT NULL);')
cur.execute('CREATE TABLE train (id serial PRIMARY KEY, link varchar(400) NOT NULL, class_id integer REFERENCES classes (id));')
cur.execute('CREATE TABLE test (id serial PRIMARY KEY, link varchar(400) NOT NULL, class_id integer REFERENCES classes (id));')
# cur.execute('DROP TABLE IF EXISTS classes;')
# cur.execute('DROP TABLE IF EXISTS train;')
# cur.execute('DROP TABLE IF EXISTS test;')

In [18]:
train_files = load_files('train', topics, section_map)
test_files = load_files('test', topics, section_map)

In [19]:
for s in section_map:
   cur.execute(f"INSERT INTO classes(id, class_name) VALUES ({s}, '{section_map[s]}');")

for file in train_files:
   cur.execute(f"INSERT INTO train(link, class_id) VALUES('{file[0]}', {file[1]});")
   
for file in test_files:
   cur.execute(f"INSERT INTO test(link, class_id) VALUES('{file[0]}', {file[1]});")

In [20]:
conn.commit()
cur.close()
conn.close()